# TikTokers (TikTok Hackathon 2023)

## Problem Set 1: Optimize Advertisement Moderation

### Part A

#### Import Libraries

In [1]:
import pandas as pd
import datetime
from sklearn.mixture import BayesianGaussianMixture
from sklearn.preprocessing import StandardScaler
import numpy as np
import simpy

#### Data Preprocessing

In [2]:
# Reading Data
path = "../data/data.xlsx"

ads = pd.read_excel(path, sheet_name='ads dimension (dim table)', header=1)
mods = pd.read_excel(path, sheet_name='moderator dimension (dim table)')

In [3]:
# ads
ads['avg_ad_revenue'].isna().sum()
max(ads['latest_punish_begin_date'])
ads['p_date_new'] = pd.to_datetime(ads['p_date'], format='%Y%m%d')
# ads
# ads['p_date'].nunique()

In [4]:
ads.ad_revenue = ads['ad_revenue'].fillna(0)
ads = ads[ads['ad_revenue'] != 0]

Fill in missing values for punish number based on inferred advertisers.

In [5]:
# ads.groupby('avg_ad_revenue')['punish_num'].fillna(ads.groupby('avg_ad_revenue')['punish_num'].agg('mode'))
# lazy grouping
groups = ads.groupby('avg_ad_revenue')

# where all the rows within a group is NaN
all_na = groups['punish_num'].transform(lambda x: x.isna().all())

# fill global mode
ads.loc[all_na, 'punish_num'] = ads['punish_num'].mode()[0]

# fill with local mode
mode_by_group = groups['punish_num'].transform(lambda x: x.mode()[0])
ads['punish_num'] = ads['punish_num'].fillna(mode_by_group)
ads['start_time'] = ads['start_time'].fillna(datetime.datetime(2000, 1, 1, 0))
# mode = ads.groupby('avg_ad_revenue')['punish_num'].agg('mode')
# mode

In [6]:
# Feature Engineering
ads['days_since_last_penalty'] = (ads['p_date_new'] - ads['latest_punish_begin_date']).dt.days
ads['days_since_start_time'] = (ads['p_date_new'] - ads['start_time']).dt.days
ads['revenue_ratio'] = (ads['ad_revenue'] / (ads['avg_ad_revenue'])).fillna(0)

In [7]:
ads.head()

,p_date,ad_id,delivery_country,queue_market,punish_num,latest_punish_begin_date,ad_revenue,avg_ad_revenue,start_time,baseline_st,product_line,task_type_en,p_date_new,days_since_last_penalty,days_since_start_time,revenue_ratio
0,20230807,1747578422390810,US,USCA,1.0,2023-07-24,5132.1700,4795.250,2022-10-24 14:36:00,1.78,Auction Ads,auction_relation,2023-08-07,14,286,1.070261
1,20230807,1758543228094480,SA,MENA,1.0,2023-06-14,1976.2500,1063.767,2023-02-22 15:50:00,1.67,RIE,Ad Group Recall (Merged),2023-08-07,54,165,1.857785
2,20230807,1738303151826990,AU,AU,2.0,2023-07-24,1.7355,5.365,2022-07-14 05:00:00,1.84,Auction Ads,auction_relation,2023-08-07,14,388,0.323486
4,20230807,1738304016410620,NZ,NZ,2.0,2023-07-24,1.5250,5.365,2022-07-14 05:17:00,1.58,Auction Ads,auction_relation,2023-08-07,14,388,0.284250
5,20230807,1738304421208090,NZ,NZ,2.0,2023-07-24,0.7889,5.365,2022-07-14 05:24:00,1.58,Auction Ads,auction_relation,2023-08-07,14,388,0.147046


In [9]:
data = ads[['ad_id', 'delivery_country', 'queue_market', 'baseline_st', 'punish_num', 'ad_revenue', 'avg_ad_revenue', 'days_since_last_penalty', 'days_since_start_time']]

In [10]:
data.values

array([[1747578422390810, 'US', 'USCA', ..., 4795.25, 14, 286],
       [1758543228094480, 'SA', 'MENA', ..., 1063.767, 54, 165],
       [1738303151826990, 'AU', 'AU', ..., 5.365, 14, 388],
       ...,
       [1773550979715070, 'VN', nan, ..., 0.4573, 8619, -1],
       [1773599824579580, 'US', nan, ..., 4.1414, 8619, -1],
       [1773551094679610, 'VN', nan, ..., 0.3121, 8619, -1]], dtype=object)

In [11]:
data.drop(columns=['delivery_country', 'queue_market']).describe()

,ad_id,baseline_st,punish_num,ad_revenue,avg_ad_revenue,days_since_last_penalty,days_since_start_time
count,2.841700e+04,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000,28417.000000
mean,1.772813e+15,1.273988,1.009537,78.417707,26.920210,8164.963156,7.184502
std,2.778871e+12,0.629223,0.217351,502.800098,172.568797,1922.045125,30.609878
min,1.714641e+15,0.540000,1.000000,0.000100,0.000000,0.000000,-1.000000
25%,1.773294e+15,0.540000,1.000000,1.804400,1.213100,8619.000000,-1.000000
50%,1.773512e+15,1.530000,1.000000,6.680000,3.340600,8619.000000,0.000000
75%,1.773559e+15,1.780000,1.000000,21.578700,10.587700,8619.000000,2.000000
max,1.773616e+15,7.590000,16.000000,27100.620000,17144.498600,8619.000000,649.000000
